In [ ]:
import pandas as pd
import numpy as np
import googlemaps

In [ ]:
def get_geocode_info(direccion,gmaps_client):
    # Consultamos dirección a googlemaps
    vector = gmaps_client.geocode(direccion, components={'country' : 'CO'}) 
    # Obtenemos latitud y longitud
    if len(vector)>0:
        coordinates = vector[0]['geometry']['location']
    else:
        coordinates={'lat' : None, 'lng' : None}

    return coordinates

In [ ]:
def save_geocode(direccion,ciudad, dic,gmaps_client):
    coordinates = get_geocode_info(direccion,gmaps_client)
    if coordinates!={'lat' : None, 'lng' : None}:
        #Si google encuentra la direccion concatenamos toda la información
        dic[direccion] = coordinates 
    else:
        #Si no, se consultan y se guardan las coordenadas de la ciudad para la direccion 
        coordinates = get_geocode_info(ciudad,gmaps_client)
        dic[direccion] = coordinates 

    return dic

In [ ]:
def compute_coordinates(locations,gmaps_client):
    locations=locations.assign(complete_address= lambda x: x.Address.astype("str") +","+ x.City.astype("str"))
    dic = {} 
    locations=locations.set_index("id")
    for idx in locations.index: 
            direccion=locations.loc[idx,"complete_address"]
            ciudad=locations.loc[idx,"City"]
            save_geocode(direccion,ciudad, dic,gmaps_client) 

    locations_geoloc= pd.DataFrame.from_dict(dic).transpose().reset_index().rename(columns={'index':'complete_address'})
    locations=locations.reset_index().merge(locations_geoloc,how="left")
    
    return locations

In [ ]:
compute_coordinates(locations,gmaps_client)